In [1]:
import pandas as pd
import collections
import numpy as np
import matplotlib.pyplot as plt
np.seterr(divide='ignore', invalid='ignore')

from sklearn.metrics import confusion_matrix, classification_report, precision_score
from sklearn.decomposition import PCA

The function for training `QDA` :
- takes the inputs : $X_{app}$ (training input matrix $n, p$), $z_{app}$ (binary class indicator matrix $n, K$), $m_{prior}$ (Gaussian prior expectation matrix $K, p$), $df_{exp}$ (Gaussian prior shrinkage parameter), $S_{prior}$ (covariance matrix $p, p, K$ for the inverse -Wishart prior), $df_{cov}$ (degree of freedom of the inverse-Wishart prior)
- provides : $pi$ (vector $K, 1$ of class proportions), $mu$ (estimated expectation matrix $K, p$), $Sig$ (estimated covariance matrices $p, p, K$)

In [2]:
def QDA(Xapp, zapp, mprior, df_exp, Sprior, df_cov):
    n, p, K =  Xapp.shape[0], Xapp.shape[1], zapp.shape[1]
    pi = np.zeros(K)
    mu = np.zeros((K, p))
    B = np.zeros((p, p, n))
    Sig = np.zeros((p, p, K))
    
    pi = np.mean(zapp, axis=0)
    for j in range(K):
        mu[j, :] = (sum(zapp[i, j] * Xapp[i, :] for i in range(n)) 
                    + df_exp * mprior[j, :]) / (sum(zapp[:, j]) + df_exp)
        
        for i in range(n):
            temp = (Xapp[i, :] - mprior[j, :]).reshape((p, -1))
            B[:, :, i] = temp @ temp.T
        
        Sig[:, :, j] = (sum((zapp[i, j] * B[:, :, i]) for i in range(n)) + 
                        df_exp * (mu[j, :] - mprior[j, :]) @ (mu[j, :] - mprior[j, :]).T + 
                        Sprior[:, :, j]) / (sum(zapp[:, j]) + df_cov + p + 2)
        
    return pi, mu, Sig

In [3]:
def QDA_without_prior(Xapp, zapp):
    n, p, K =  Xapp.shape[0], Xapp.shape[1], zapp.shape[1]
    pi = np.zeros(K)
    mu = np.zeros((K, p))
    B = np.zeros((p, p, n))
    Sig = np.zeros((p, p, K))
    
    pi = np.mean(zapp, axis=0)
    for j in range(K):
        mu[j, :] = sum(zapp[i, j] * Xapp[i, :] for i in range(n))
        
        for i in range(n):
            temp = (Xapp[i, :] - mu[j, :]).reshape((p, -1))
            B[:, :, i] = temp @ temp.T
        
        Sig[:, :, j] = sum((zapp[i, j] * B[:, :, i]) for i in range(n))
        
    return pi, mu, Sig

The function for evaluating test set `evaluation` :
- takes the inputs : $X_{tst}$ (test input matrix $n, p$), $pi$ (vector $K, 1$ of class proportions), $mu$ (estimated expectation matrix $K, p$), $Sig$ (estimated covariance matrices $p, p, K$)
- provides : $prob$ (matrix $n, K$ of estimated class posterior probabilities) and $pred$ (vector $n, 1$ of corresponding decisions)

In [4]:
def evaluation(Xtst, pi, mu, Sig):
    n, p, K = Xtst.shape[0], Xtst.shape[1], len(pi)
    prob = np.zeros((n, K))
    pred = np.zeros(n)
    
    f = lambda x, mu, Sig: (2 * np.pi)**(-n/2) * np.linalg.det(Sig)**(-1/2) * np.exp((-1/2) * 
                                                                                     (x - mu).T
                                                                                     @ np.linalg.inv(Sig)
                                                                                     @ (x - mu))
   
    for j in range(K):
        for i in range(n):
            total = sum(pi[k] * f(Xtst[i, :], mu[k, :], Sig[:, :, k]) for k in range(K))
            prob[i, j] = pi[j] * f(Xtst[i, :], mu[j, :], Sig[:, :, j]) / total
        
    pred = np.argmax(prob, axis=1)
    
    return prob, pred

We define a `data_processing` function to test different dataset. The function does respectively :
- Read the dataset by the `pandas` library and transform it to the one in form of the `numpy` library in order to facilitate our next works.
- Apply PCA filtering process to the dataset if necessary.
- Sort all classes of the dataset and display their enumeration
- Separate the dataset into training and test ones by `cut_off` coefficient predefined at `0.9`
- Calculate the prior expectation and variance based on the training set

In [5]:
def data_processing(file_url, n_components_pca=None, cut_off=0.9):
    data = pd.read_csv(file_url, header=0)
    data = data.to_numpy()
    
    if n_components_pca is not None:
        data_y = data[:, -1]
        data_X = data[:, :-1]
        pca = PCA(n_components=n_components_pca)
        data_X = pca.fit_transform(data_X)
        data = np.hstack((data_X, data_y.reshape((-1,1))))
    
    classes = sorted(collections.Counter(list(data[:, -1])).keys())
    classes_with_index = {}
    
    K = len(classes)
    n, p = data.shape[0], data.shape[1] - 1
    for i_class in range(K):
        classes_with_index[i_class] = classes[i_class]
        data[data[:, -1] == classes[i_class], -1] = i_class
    print("Ours classes are enumerated as bellow : \n", classes_with_index)
    
    cut_off = int(cut_off*data.shape[0])
    if cut_off > data.shape[0]:
        raise IndexError("list index out of range")
    else:
        Xapp = data[0:cut_off, :-1]
        Xtst = data[cut_off:, :-1]

        Zapp = np.zeros((cut_off, K))
        Ztst = np.zeros((n - cut_off, K))

        for i in range(cut_off):
            Zapp[i, int(data[i, -1])] = 1
        for i in range(cut_off, n):
            Ztst[i - cut_off, int(data[i, -1])] = 1
    
    mprior = np.zeros((K, p))
    Sprior = np.zeros((p, p, K))
    for j in range(K):
        temp = data[data[:, -1] == j, :-1]
        temp = temp.astype(float)
        mprior[j, :] = temp.mean(axis = 0)
        Sprior[:, :, j] = np.cov(temp, rowvar=False)
    return Xapp, Zapp, Xtst, Ztst, mprior, Sprior

The `optdigits` dataset has many zero elements that make itselft correlated. That's why the result from the inital training datatset is badly evaluated. We then apply a PCA filtering process with number of principal components depending on the `cut_off` coefficient (rank of training set) in order to eliminate the correlated ones. As a result, we get much better performance. <br>
For the choice of the shrinkage parameter $df_{exp}$ and the degree of freedom $df_{co}v$, we a priori choose $df_{exp} = 10$ and $df_{cov} = 300$ for this dataset and also the next ones.

In [6]:
df_exp_opt = 10
df_cov_opt = 300
#cuts_off = np.array([0.9, 0.91, 0.92, 0.93, 0.94, 0.95])
#prediction = np.zeros(4)

#for i in range(len(cuts_off))
Xapp_opt, Zapp_opt, Xtst_opt, Ztst_opt, mprior_opt, Sprior_opt = data_processing(
                                            "dataset/optdigits.csv", n_components_pca=10, cut_off=0.9)
pi_opt, mu_opt, Sig_opt = QDA(Xapp_opt, Zapp_opt, mprior_opt, df_exp_opt, Sprior_opt, df_cov_opt)
prob_opt, pred_opt = evaluation(Xtst_opt, pi_opt, mu_opt, Sig_opt)
#prediction[i] = precision_score(np.argmax(Ztst_opt, axis=1), pred_opt, average='weighted')

print('\n\n', confusion_matrix(np.argmax(Ztst_opt, axis=1), pred_opt))
print('\n\n', classification_report(np.argmax(Ztst_opt, axis=1), pred_opt))
print('\n\n', precision_score(np.argmax(Ztst_opt, axis=1), pred_opt, average='weighted'))

#plt.plot(cuts_off, prediction)

Ours classes are enumerated as bellow : 
 {0: 0.0, 1: 1.0, 2: 2.0, 3: 3.0, 4: 4.0, 5: 5.0, 6: 6.0, 7: 7.0, 8: 8.0, 9: 9.0}


 [[54  0  0  0  1  0  0  0  0  0]
 [ 0 53  0  0  0  0  0  0  0  5]
 [ 0  0 54  0  0  0  0  0  0  0]
 [ 0  0  0 50  0  2  0  2  3  0]
 [ 0  0  0  0 57  0  0  1  1  0]
 [ 0  0  0  0  0 55  0  0  1  0]
 [ 0  0  0  0  0  0 56  0  1  0]
 [ 0  0  0  0  0  0  0 57  0  0]
 [ 0  4  0  1  0  0  0  1 41  6]
 [ 0  0  0  1  0  2  0  0  0 53]]


               precision    recall  f1-score   support

           0       1.00      0.98      0.99        55
           1       0.93      0.91      0.92        58
           2       1.00      1.00      1.00        54
           3       0.96      0.88      0.92        57
           4       0.98      0.97      0.97        59
           5       0.93      0.98      0.96        56
           6       1.00      0.98      0.99        57
           7       0.93      1.00      0.97        57
           8       0.87      0.77      0.82        53

In [7]:
pi_opt_2, mu_opt_2, Sig_opt_2 = QDA_without_prior(Xapp_opt, Zapp_opt)
prob_opt_2, pred_opt_2 = evaluation(Xtst_opt, pi_opt_2, mu_opt_2, Sig_opt_2)
#prediction[i] = precision_score(np.argmax(Ztst_opt, axis=1), pred_opt, average='weighted')

print('\n\n', confusion_matrix(np.argmax(Ztst_opt, axis=1), pred_opt_2))
print('\n\n', classification_report(np.argmax(Ztst_opt, axis=1), pred_opt_2))
print('\n\n', precision_score(np.argmax(Ztst_opt, axis=1), pred_opt_2, average='weighted'))

#plt.plot(cuts_off, prediction)



 [[55  0  0  0  0  0  0  0  0  0]
 [58  0  0  0  0  0  0  0  0  0]
 [54  0  0  0  0  0  0  0  0  0]
 [57  0  0  0  0  0  0  0  0  0]
 [59  0  0  0  0  0  0  0  0  0]
 [56  0  0  0  0  0  0  0  0  0]
 [57  0  0  0  0  0  0  0  0  0]
 [57  0  0  0  0  0  0  0  0  0]
 [53  0  0  0  0  0  0  0  0  0]
 [56  0  0  0  0  0  0  0  0  0]]


               precision    recall  f1-score   support

           0       0.10      1.00      0.18        55
           1       0.00      0.00      0.00        58
           2       0.00      0.00      0.00        54
           3       0.00      0.00      0.00        57
           4       0.00      0.00      0.00        59
           5       0.00      0.00      0.00        56
           6       0.00      0.00      0.00        57
           7       0.00      0.00      0.00        57
           8       0.00      0.00      0.00        53
           9       0.00      0.00      0.00        56

    accuracy                           0.10       562
   macro avg 

/home/huydung/miniconda3/envs/aos1/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/huydung/miniconda3/envs/aos1/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [8]:
Xapp_pag, Zapp_pag, Xtst_pag, Ztst_pag, mprior_pag, Sprior_pag = data_processing("dataset/page-blocks.csv")

df_exp_pag = 10
df_cov_pag = 300

pi_pag, mu_pag, Sig_pag = QDA(Xapp_pag, Zapp_pag, mprior_pag, df_exp_pag, Sprior_pag, df_cov_pag)
prob_pag, pred_pag = evaluation(Xtst_pag, pi_pag, mu_pag, Sig_pag)

print('\n\n')
print(confusion_matrix(np.argmax(Ztst_pag, axis=1), pred_pag))
print('\n\n')
print(classification_report(np.argmax(Ztst_pag, axis=1), pred_pag))
print('\n\n')
print(precision_score(np.argmax(Ztst_pag, axis=1), pred_pag, average='weighted'))

Ours classes are enumerated as bellow : 
 {0: 1.0, 1: 2.0, 2: 3.0, 3: 4.0, 4: 5.0}



[[428   5   0   2  15]
 [ 13  59   0   0   0]
 [  2   0   2   0   0]
 [  2   0   0  15   0]
 [  2   0   0   0   3]]



              precision    recall  f1-score   support

           0       0.96      0.95      0.95       450
           1       0.92      0.82      0.87        72
           2       1.00      0.50      0.67         4
           3       0.88      0.88      0.88        17
           4       0.17      0.60      0.26         5

    accuracy                           0.93       548
   macro avg       0.79      0.75      0.73       548
weighted avg       0.94      0.93      0.93       548




0.9435781323992881


In [9]:
Xapp_sat, Zapp_sat, Xtst_sat, Ztst_sat, mprior_sat, Sprior_sat = data_processing("dataset/satimage.csv")

df_exp_sat = 10
df_cov_sat = 300

pi_sat, mu_sat, Sig_sat = QDA(Xapp_sat, Zapp_sat, mprior_sat, df_exp_sat, Sprior_sat, df_cov_sat)
prob_sat, pred_sat = evaluation(Xtst_sat, pi_sat, mu_sat, Sig_sat)

print('\n\n')
print(confusion_matrix(np.argmax(Ztst_sat, axis=1), pred_sat))
print('\n\n')
print(classification_report(np.argmax(Ztst_sat, axis=1), pred_sat))
print('\n\n')
print(precision_score(np.argmax(Ztst_sat, axis=1), pred_sat, average='weighted'))

Ours classes are enumerated as bellow : 
 {0: 1.0, 1: 2.0, 2: 3.0, 3: 4.0, 4: 5.0, 5: 7.0}



[[164   0   1   0   3   0]
 [  0  63   0   0   1   0]
 [  1   2 134   1   1   1]
 [  1   3  20  12   5  28]
 [  1   2   0   0  64   2]
 [  0   3   5   6   5 114]]



              precision    recall  f1-score   support

           0       0.98      0.98      0.98       168
           1       0.86      0.98      0.92        64
           2       0.84      0.96      0.89       140
           3       0.63      0.17      0.27        69
           4       0.81      0.93      0.86        69
           5       0.79      0.86      0.82       133

    accuracy                           0.86       643
   macro avg       0.82      0.81      0.79       643
weighted avg       0.84      0.86      0.83       643




0.8421588047083794


In [10]:
Xapp_seg, Zapp_seg, Xtst_seg, Ztst_seg, mprior_seg, Sprior_seg = data_processing("dataset/segment.csv",
                                                                                n_components_pca=17)

df_exp_seg = 10
df_cov_seg = 300

pi_seg, mu_seg, Sig_seg = QDA(Xapp_seg, Zapp_seg, mprior_seg, df_exp_seg, Sprior_seg, df_cov_seg)
prob_seg, pred_seg = evaluation(Xtst_seg, pi_seg, mu_seg, Sig_seg)

print('\n\n')
print(confusion_matrix(np.argmax(Ztst_seg, axis=1), pred_seg))
print('\n\n')
print(classification_report(np.argmax(Ztst_seg, axis=1), pred_seg))
print('\n\n')
print(precision_score(np.argmax(Ztst_seg, axis=1), pred_seg, average='weighted'))

Ours classes are enumerated as bellow : 
 {0: 'brickface', 1: 'cement', 2: 'foliage', 3: 'grass', 4: 'path', 5: 'sky', 6: 'window'}



[[25  1  0  0  0  0  2]
 [ 0 37  0  0  0  0  2]
 [ 0  0 16  0  0  0 12]
 [ 0  0  0 29  0  0  0]
 [ 0  4  0  0 31  0  0]
 [ 0  0  0  0  0 38  0]
 [ 0 11  1  0  0  0 22]]



              precision    recall  f1-score   support

           0       1.00      0.89      0.94        28
           1       0.70      0.95      0.80        39
           2       0.94      0.57      0.71        28
           3       1.00      1.00      1.00        29
           4       1.00      0.89      0.94        35
           5       1.00      1.00      1.00        38
           6       0.58      0.65      0.61        34

    accuracy                           0.86       231
   macro avg       0.89      0.85      0.86       231
weighted avg       0.88      0.86      0.86       231




0.879928860593619


In [11]:
# 0.9766310415063318 1;p+20 ll
# 0.9758926103474012 10;p+20
# 0.9671482548426473 10;p+300 
# v reprensent the least informative -> v high -> cov lower -> more "confident" ????
# high df_cov -> Sig =~~ sampling covariance matrix (in this case Sprior)